In [1]:
#import
import pandas as pd
import matplotlib.pyplot as plt
import tweepy
import json
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

from config import api_twitter
from config import api_twitter_secret
from config import api_access_token
from config import api_access_token_secret
# Your Twitter API Keys
consumer_key = api_twitter
consumer_secret = api_twitter_secret
access_token = api_access_token
access_token_secret = api_access_token_secret

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())


In [34]:
#Let's keep our twitter targets in a list
target_list = ["@bbc", "@cnn"]
comp_dict = {}
pos_dict = {}
neg_dict = {}
neu_dict = {}

In [35]:
#Just to make sure it's working....
#tweets = api.user_timeline(target_list[0])
#print(tweets)
#tweets[0]["text"]

In [40]:
'''Your final Jupyter notebook must:

Pull last 100 tweets from each outlet.
Perform a sentiment analysis with the compound, positive, neutral, and negative scoring for each tweet.
Pull into a DataFrame the tweet's source acount, its text, its date, and its compound, positive, neutral, and negative sentiment scores.
Export the data in the DataFrame into a CSV file.
Save PNG images for each plot.'''


#let's create a function series of functions to do this.
def compoundanalysis(handle):
    print("Running compound analysis for {}".format(handle))
    compound_list = []
    oldest_tweet = None
    for x in range (0,5):
        tweet_search = api.user_timeline(handle, max_id = oldest_tweet)
        for tweet in tweet_search:
            results = analyzer.polarity_scores(tweet["text"])
            compound_list.append(results["compound"])
            oldest_tweet = tweet["id"]
    return(compound_list)

#Damn, ok, that wasn't so bad. That return really opens up some good stuff. Let's create 3 more for other returns
    
def positiveanalysis(handle):
    print("Running positive analysis for {}".format(handle))
    positive_list = []
    oldest_tweet = None
    for x in range (0,5):
        tweet_search = api.user_timeline(handle, max_id = oldest_tweet)
        for tweet in tweet_search:
            results = analyzer.polarity_scores(tweet["text"])
            positive_list.append(results["pos"])
            oldest_tweet = tweet["id"]
    return(positive_list)

def negativeanalysis(handle):
    print("Running negative analysis for {}".format(handle))
    negative_list = []
    oldest_tweet = None
    for x in range (0,5):
        tweet_search = api.user_timeline(handle, max_id = oldest_tweet)
        for tweet in tweet_search:
            results = analyzer.polarity_scores(tweet["text"])
            negative_list.append(results["neg"])
            oldest_tweet = tweet["id"]
    return(negative_list)

def neutralanalysis(handle):
    print("Running neutral analysis for {}".format(handle))
    neutral_list = []
    oldest_tweet = None
    for x in range (0,5):
        tweet_search = api.user_timeline(handle, max_id = oldest_tweet)
        for tweet in tweet_search:
            results = analyzer.polarity_scores(tweet["text"])
            neutral_list.append(results["neu"])
            oldest_tweet = tweet["id"]
    return(neutral_list)

In [ ]:
#Then we just do this, and build a dictionary! Ain't no thing.
for handle in target_list:
    comp_dict["comp_" + handle] = compoundanalysis(handle)
    pos_dict["pos_"+handle] = positiveanalysis(handle)
    neg_dict["neg_"+handle] = negativeanalysis(handle)
    neu_dict["neu_"+handle] = neutralanalysis(handle)

Running compound analysis for @bbc
Running positive analysis for @bbc
Running negative analysis for @bbc
Running neutral analysis for @bbc
Running compound analysis for @cnn
Running positive analysis for @cnn


In [31]:
#Turn it all into a dataframe by taking a list of those dictionaries
tweet_df = py.DataFrame([comp_dict],[pos_dict],[neg_dict],[neu_dict])
tweet_df.head()

100

In [23]:
len([0.0, 0.4939, 0.25, 0.0, 0.0, -0.2263, 0.0, 0.0, -0.2023, 0.875, 0.2732, -0.0772, 0.5859, 0.3818, 0.0, 0.0, 0.6369, 0.3612, 0.5719, 0.0, 0.0, -0.7269, 0.6515, 0.6908, 0.0, 0.0, -0.5719, -0.296, 0.0, 0.0, 0.69, 0.0, 0.0, -0.8658, 0.0, 0.34, 0.4588, 0.0, 0.0, 0.0, 0.0, 0.0, -0.1531, -0.7777, 0.6037, 0.8126, 0.0, 0.0, 0.5499, -0.5849, 0.0, -0.4215, 0.0, 0.2023, 0.8617, 0.0, 0.0, 0.0, 0.0, 0.25, 0.25, 0.0, 0.4579, -0.6908, 0.7096, 0.0, -0.765, 0.0, 0.0, 0.0028, 0.4588, -0.2263, 0.0, 0.0, 0.4215, 0.6027, 0.0, 0.2023, 0.3164, 0.0])

80